# Projet - Projectile 1

---

Dans ce projet, nous visons à appliquer des techniques d'apprentissage automatique supervisé à un problème classique en physique : prédire la distance horizontale parcourue par un projectile. L'objectif est de construire des modèles prédictifs pour deux tâches liées mais distinctes. 

- Pour le Projet 1, l'objectif est de prédire la distance horizontale en se basant sur les composantes de la vitesse initiale le long des axes x et y ($v_x$ et $v_y$). 
- Pour le Projet 2, le modèle prédira la distance horizontale en utilisant la magnitude de la vitesse initiale ($v$) et l'angle de lancement ($\theta$). 

Les deux tâches impliquent l'entraînement de modèles de régression sur des données de projectile simulées, avec pour objectif ultime de capturer précisément les relations physiques sous-jacentes à partir de données d'exemple. Nous verrons qu'un système donné, lorsqu'il est entraîné pour différents problèmes, montre la flexibilité de s'adapter à différentes situations.


## 1. Vue d'ensemble

Rappel des ingrédients clés de l'apprentissage automatique supervisé :

- Tâche (T)
- Expérience (E)
- Mesure de performance (P)
- Espace d'hypothèses (Modèle d'apprentissage automatique)
- Algorithme d'apprentissage
- Généralisation

Nous passerons en revue tous ces éléments en détail avec le Projet 1, puis appliquerons ces concepts et techniques au Projet 2

## 2. Projet-1 : distance horizontale à partir des composantes de la vitesse initiale ($v_x$ & $v_y$)

### 2.1 Définir la tâche pour l'apprentissage automatique via la "_fonction cible_"

En apprentissage automatique supervisé, l'objectif est toujours d'inférer à partir des données ("expérience") la relation entre deux ensembles de variables appelés "**caractéristiques**" et "**étiquettes**" (également appelées "**cibles**") d'un sujet. La **caractéristique** et l'**étiquette** peuvent toutes deux être composées de plusieurs quantités ou variables, où chaque variable représente une propriété du sujet. 

> Dans le projet actuel, 
> 
> - le "sujet" étudié est le projectile lancé sous l'effet de la gravité, 
> - les "caractéristiques" sont la paire de composantes de la vitesse de lancement $(v_x,\; v_y)$, et 
> - l'"étiquette" est la distance horizontale (c'est-à-dire le long de $x$) de la position d'atterrissage du projectile par rapport au point de lancement, notée $d$.

La tâche destinée à un système d'apprentissage supervisé est de retourner aussi précisément que possible l'**étiquette** lorsqu'une **caractéristique** comprimant un ensemble de variables pré-conventionnées est fournie. Ainsi, du point de vue de la machine, les **caractéristiques** sont alternativement appelées "**entrée**" et l'**étiquette** est alternativement appelée "**sortie**". 

Le mapping de la **caractéristique** vers l'**étiquette** pour un sujet dans le monde réel est la **fonction cible**. C'est l'_association réelle d'une étiquette à certaines caractéristiques_, c'est-à-dire la **fonction cible**, qui doit être apprise par une machine. 

La **fonction cible**, notée $f_T(\cdot)$, est spécifiée par 

- la forme de la "caractéristique" (ou "entrée") -- le _domaine de définition_ et la signification de chacune de ses variables.
- la forme de l'"étiquette" (ou "sortie") -- le _domaine de définition_ et la signification de chacune de ses variables. 

> Dans le projet actuel, la fonction cible $f_T(\cdot)$ est spécifiée par 
> - le domaine des caractéristiques $X\hat{=}\{ (v_x, v_y) | v_x \in \mathbb{R}^+, \; v_y \in \mathbb{R}^+ \}$ où $v_x$ et $v_y$ sont respectivement les composantes horizontale et verticale de la vitesse de lancement, et 
> - le domaine des étiquettes $Y\hat{=}\{d|d\in \mathbb{R}^+\}$ où $d$ représente la distance d'atterrissage du projectile.
> 
> La fonction cible est formellement  $$ f_T:X\rightarrow Y \quad \text{ou} \quad f_T(v_x, v_y) = d$$
>
> Dans la majorité des situations, contrairement au problème de projectile actuel où la fonction cible peut être résolue (en utilisant la physique), la fonction cible est trop complexe pour être résolue, et la tâche de l'apprentissage automatique supervisé est d'inférer cette **fonction cible** inconnue en utilisant certaines techniques avec les données disponibles.   

**_Spécifier la fonction cible définit la tâche destinée à un système d'apprentissage automatique_**. Cela conduit à des indications cruciales pour 

1. Le pipeline de données : l'ensemble du processus depuis la collecte de données brutes jusqu'à la formation d'ensembles de données d'entraînement et de test prêts pour l'entraînement et le test de modèles d'apprentissage automatique. 
2. L'espace d'hypothèses : la portée des modèles d'apprentissage automatique candidats à utiliser, c'est-à-dire les modèles qui peuvent mapper du domaine des caractéristiques $X$ vers le domaine des étiquettes $Y$.
3. La définition de la mesure de performance : lorsqu'une cible $\hat t$ produite par un système d'apprentissage automatique ne correspond pas à la cible réelle $t$, il faut spécifier la **fonction de perte**, notée $L(\cdot, \cdot)$ qui mappe $(t,\hat t)\in Y^2$ vers un domaine scalaire généralement $\mathbb{R}^+$. 

### 2.2 Exploration des données

Une fois que la fonction cible est bien spécifiée, le produit final du pipeline de données est également clarifié, c'est-à-dire un ensemble de paires "caractéristique-cible" observées. En pratique, si aucune donnée brute n'est fournie, il faut concevoir le processus de collecte et de nettoyage des données afin de produire les paires "caractéristique-cible" prêtes à l'emploi, ou bien transformer les données brutes sous la forme de paires "caractéristique-cible" requises par la fonction cible. 

Dans ce projet actuel, le produit final du pipeline de données, c'est-à-dire les paires caractéristique-cible, est préparé dans `/training_set_1.dat` pour que vous puissiez poursuivre les étapes d'apprentissage automatique.

Une fois que l'ensemble de données d'entraînement de paires caractéristique-cible est prêt, il est utile d'effectuer l'exploration des données pour obtenir des informations sur les connexions entre toutes les variables (dans les caractéristiques et la cible) afin d'orienter judicieusement le choix des modèles d'apprentissage automatique. 

L'**exploration des données**, en principe, devrait être dirigée par des questions génériques sur le mécanisme interne sous-jacent au sujet, qui varie selon les cas et les approches. Ici, nous passerons en revue quelques procédures courantes d'exploration des données à travers une série d'exercices et dériverons quelques informations pour choisir les modèles d'apprentissage automatique pour le projet actuel.

#### **Exercice 2.2.1** Charger les données

Chargez les fichiers de données suivants avec la fonction `numpy.loadtxt` :
  - "training_set_1.dat"
  - "test_set_1.dat" 

1. Explorez la structure des données chargées. Combien d'entrées de paires "caractéristique-cible" y a-t-il dans chaque ensemble de données ?
2. Explorez l'en-tête des fichiers de données et déterminez quelles colonnes sont les entrées (caractéristiques) et quelles colonnes sont la sortie (cibles) ?

Utilisez la cellule de code ci-dessous. Rappel : vous pouvez utiliser `help()`, `dir()` et `type()` pour le manuel des nouveaux objets en Python.

In [ ]:
import numpy as np

# Charger les fichiers de données
# train1 = np.loadtxt(..)

# structure des données, combien de colonnes et de lignes ?

# en-tête des fichiers de données, quelles colonnes sont les entrées (caractéristiques) et lesquelles sont la sortie (cibles) ?
header_lines = []
with open('path_to_file', 'r') as f:
    for line in f:
        if line.startswith('#'):
            header_lines.append(line)
    
print(header_lines)

# afficher les 5 premières entrées de l'ensemble d'entraînement et de l'ensemble de test    


#### **Exercice 2.2.2** Distribution des variables d'entrée

Avec l'aide de `matplotlib`, explorez les aspects suivants des variables d'entrée (variables de caractéristiques) dans `training_set_1.dat` :

1. Pour chaque variable dans la caractéristique, quelle est sa distribution empirique ? Indice : on peut tracer l'histogramme en utilisant `matplotlib.pyplot.hist`.
2. Y a-t-il une valeur la plus probable que chaque variable d'entrée peut prendre ?
3. Estimez l'espérance de chaque variable d'entrée.
4. Estimez la fluctuation de chaque variable d'entrée autour de son espérance.
5. Comment transformer une variable d'entrée de sorte qu'elle ait une espérance nulle et un écart-type unitaire ? Une telle transformation est appelée "normalisation".

Utilisez la cellule de code suivante pour cet exercice.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget  # for interactive plotting

# load the training set and declare input variables
# train1 = np.loadtxt(..)
# vx = 
# vy = 

# Distribution of the input variables by ploting the histogram of vx and vy
# plt.hist(..)

# Estimate the expectation

# Estimate the fluctuation

# Transform the input variable such that it has zero expectation and unity standard deviation

#### **Exercice 2.2.3** Corrélation entre les variables d'entrée

Pour les mêmes variables d'entrée étudiées dans l'exercice précédent, ces variables d'entrée sont-elles corrélées ? La valeur d'une variable d'entrée est-elle informative pour la valeur d'autres variables d'entrée ? Indice : on peut tracer une variable en fonction d'une autre pour révéler un signe de dépendance mutuelle.

Utilisez la cellule de code suivante pour l'investigation.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget  # for interactive plotting

# Correlation 

#### **Exercice 2.2.4** Distribution des valeurs cibles

Toujours avec les données dans `training_set_1.dat`, nous nous tournons maintenant vers l'investigation des propriétés statistiques des variables cibles. En utilisant la même technique, explorez les aspects suivants des variables cibles

1. La distribution empirique de la variable cible.
2. Y a-t-il une valeur la plus probable pour la variable cible ?
3. Estimez l'espérance.
4. Estimez la fluctuation.
5. Normalisez les variables cibles.

Utilisez la cellule de code suivante pour cet exercice.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget  # pour affichage interactif

# Distribution des valeurs cibles

# La valeur la plus probable pour la variable cible

# Estimation de l'espérance

# Estimation de la fluctuation

# Normalisation des variables cibles


#### **Exercice 2.2.5** Comment la cible dépend-elle des variables d'entrée

Maintenant, nous nous tournons vers l'investigation de la façon dont la cible dépend des variables d'entrée dans `training_set_1.dat`.

1. Pour chaque variable d'entrée, explorez comment la variable cible dépend de la variable d'entrée en utilisant des graphiques. 
2. Calculez le coefficient de corrélation entre la variable cible et chacune des variables d'entrée.
3. Résumez vos résultats pour Q1 et Q2.
4. Comment révéler la dépendance de la cible par rapport aux deux variables d'entrée ? Indice : faites un graphique de dispersion où la position de chaque point représente les entrées et utilisez sa couleur pour la cible.
5. Quelles sont vos observations de Q4 ? Quelle forme pouvez-vous deviner pour la fonction cible ?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget  # pour affichage interactif

# Nuage de points de la cible par rapport à chacune des variables d'entrée

# Nuage de points de la cible par rapport aux deux variables d'entrée


### 2.3 Espace d'hypothèses, métrique de performance et algorithme d'apprentissage

En une phrase, l'**algorithme d'apprentissage** recherche, dans un domaine défini par l'**espace d'hypothèses**, la fonction (également appelée « hypothèse » ou « modèle ») optimale pour approximer la fonction cible selon la **métrique de performance**.

Nous passerons en revue les concepts clés un par un.

- Un **espace d'hypothèses**, noté $\mathcal{H}$, définit un ensemble de fonctions possibles (ou modèles) $h(\cdot)$ parmi lesquelles une "optimale" peut être choisie pour effectuer la tâche de la fonction cible $f_T$, c'est-à-dire mapper chaque caractéristique dans $X$ vers une cible dans $Y$. Par exemple, pour une fonction cible $f_T:\mathbb{R}\rightarrow \mathbb{R}$, on peut proposer un espace d'hypothèses $$\mathcal{H}=\{h(x)=ax^{p}| a\in \mathbb{R}, p\in\mathbb{Z}\}\quad .$$ 

  En effet, assez souvent comme dans cet exemple, l'**espace d'hypothèses** peut être vu comme un ensemble de fonctions d'une forme spécifique avec des paramètres variables, et l'**espace d'hypothèses** est équivalemment représenté par l'espace de tous les réglages de paramètres possibles. Dans le dernier exemple, $\mathcal{H} \leftrightarrow \{(a,p)|(a,p)\in \mathbb{R}\times\mathbb{Z}\} \;(= \mathbb{R}\times\mathbb{Z})$.

- La fonction "optimale" $h^*(\cdot)$ (dans le cadre de $\mathcal{H}$) est choisie contre une **métrique de performance** habituelle qui quantifie à quel point une fonction $h(\cdot)$ approxime la fonction cible $f_T(\cdot)$. Ce concept comprend deux ingrédients :
  - **fonction de perte**
  - Perte sur la population -- **Perte attendue**.

- La **fonction de perte**, notée $L$, associe un degré de "perte", c'est-à-dire un scalaire, à une paire de la cible réelle $y = f_T(x)$ et de la cible prédite $\hat{y} = h(x)$. Formellement $L(\hat{y}, y):Y^2\rightarrow \mathbb{R}$. La fonction de perte indique essentiellement à quel point c'est mauvais si une cible prédite est $\hat y$ alors que la cible réelle est $y$. En général, vous voulez une hypothèse qui donne une petite valeur de la fonction de perte. Voici deux exemples courants de fonctions de perte $L(\hat y, y)=|\hat y - y|$ et $L(\hat y, y)=(\hat y - y)^2$ (pour le domaine cible $Y=\mathbb{R}$).

#### **Exercice 2.3.1 : Comparer deux fonctions**

En supposant que la fonction cible $f_T:\mathbb{R}\rightarrow \mathbb{R}$, comment comparer les deux fonctions suivantes $h_1(x)=2x$ et $h_2(x)=2x^2$ en utilisant la même métrique de performance $L(\hat y, y)=(\hat y -y)^2$ ? 

Un échantillon de paires caractéristique-cible est collecté à partir d'une certaine population-1 et stocké dans le fichier `population_1.dat`. 

1. Combien de paires caractéristique-cible y a-t-il dans cet échantillon ?
2. Construisez un tableau numpy de cibles prédites par $h_1$ et un tableau numpy de cibles prédites par $h_2$. Nommez ces deux tableaux `y1` et `y2` respectivement.
3. Construisez un graphique de dispersion de la perte de $h_1$ en fonction des caractéristiques collectées. Faites de même pour $h_2$ sur la même figure.
4. Selon le graphique de dispersion dans Q3, $h_1$ surpasse-t-elle ou sous-performe-t-elle toujours $h_2$ ?
5. Quelle est la distribution empirique de la caractéristique dans cette population ?
6. En tenant compte de la distribution des caractéristiques, pouvez-vous deviner quelle fonction, $h_1$ ou $h_2$, performe mieux sur l'ensemble de la population ?
7. Proposez une mesure qui quantifie la performance d'une fonction $h$ sur l'ensemble de la population par rapport à une fonction de perte. Appliquez cette mesure à $h_1$ et $h_2$ avec la perte $L(\hat y, y)=(\hat y - y)^2$. Imprimez le résultat, laquelle est meilleure ?

Utilisez la cellule de code suivante pour cet exercice.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.clf()
%matplotlib widget

xs = np.linspace(0, 10, 101)

h1 = lambda x: 2*x
h2 = lambda x: 2*x**2

loss_func = lambda y_true, y_pred: (y_true - y_pred)**2

d1 = np.loadtxt('./data/projectile_1/population_1.dat')

# combien de paires caractéristique-cible y a-t-il dans cet échantillon ?

# construisez les tableaux numpy des cibles prédites par h1 et h2
# y1 = 
# y2 = 

# graphique de dispersion de la perte de h1 en fonction des caractéristiques collectées dans la population 1

# graphique de dispersion de la perte de h2 en fonction des caractéristiques collectées dans la population 1

# histogramme de la caractéristique dans cette population

# affichez le résultat de la mesure de performance pour h1 et h2

#### **Exercice 2.3.2 : Comparaison à nouveau pour une population différente**

Maintenant, nous investiguons une population différente `population_2.dat`. Il est donné que la population 1 et la population 2 admettent la même fonction cible. Comparez à nouveau la performance de $h_1$ et $h_2$ mais cette fois sur la population 2. 

1. Quelle est la distribution de la caractéristique dans la population 2 ?
2. Comparez les deux populations en termes de distribution des caractéristiques.
3. Devinez sur l'ensemble de la population 2, laquelle, $h_1$ ou $h_2$, performera mieux ?
4. Vérifiez votre supposition en utilisant la mesure définie dans l'exercice précédent Q7.
5. Quelle conclusion pouvez-vous tirer sur la façon de comparer correctement la performance de différentes fonctions ? 

Utilisez la cellule de code suivante.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.clf()
%matplotlib widget

xs = np.linspace(0, 10, 101)

h1 = lambda x: 2*x
h2 = lambda x: 2*x**2

loss_func = lambda y_true, y_pred: (y_true - y_pred)**2

d2 = np.loadtxt('./data/projectile_1/population_2.dat')

# quelle est la distribution de la caractéristique dans la population 2 ?

# comparez les deux populations en termes de distribution de la caractéristique, moyenne, écart-type, etc.

# vérifiez votre supposition en utilisant la mesure définie dans l'exercice précédent Q7.


- **Perte attendue** et **Perte empirique** -- La fonction de perte n'assigne qu'un degré de mauvais résultat à une instance de paire caractéristique-cible lorsqu'une hypothèse (fonction) $h$ est appliquée. Cependant, nous voulons optimiser notre fonction (dans l'espace d'hypothèses) de sorte qu'elle performe bien sur toutes les caractéristiques possibles rencontrées. C'est pourquoi nous introduisons la **Perte attendue** pour évaluer la "mauvaiseté" d'une fonction $h$ sur l'ensemble de la population de caractéristiques. Puisqu'on ne dispose que des données observées (c'est-à-dire les données d'entraînement) pour acquérir des connaissances sur la population, on doit utiliser la **perte empirique** définie comme $$ \mathcal{L} = \frac{1}{n}\sum_{i=1}^n L(h(x_i), y_i)$$ où $(x_1,y_1),(x_2, y_2),\ldots,(x_n,y_n)$ sont des paires caractéristique-cible observées, pour estimer la **perte attendue**. 

  **Remarque**
   - Il est cependant important d'être clair que la **perte empirique**, qui estime la performance dans l'échantillon, n'est pas la **perte attendue**, qui mesure la performance sur l'ensemble de la population. La seule façon de rendre ces deux quantités égales est de faire tendre $n$ vers l'infini (étant donné que les paires caractéristique-cible sont générées indépendamment). 
   - Cette remarque a des implications importantes sur la façon dont une fonction $h$ optimisée sur les échantillons d'entraînement peut performer hors de l'échantillon, c'est-à-dire la perte attendue. Lorsque l'optimisation est trop poussée pour optimiser la **perte empirique**, elle peut aller à l'encontre de la généralisation de sorte que la **perte attendue** n'est pas optimale. Une technique appelée **régularisation** est introduite pour ce problème, qui sera discutée plus tard.

- La **perte empirique** comme un _paysage_. Pour un ensemble donné de données observées (c'est-à-dire en fixant les paires caractéristique-cible), la **perte empirique** (ainsi que la **perte attendue**) définit une fonction multivariée mappant les paramètres d'hypothèse vers un scalaire, qui peut être vue comme une hyper-surface ou un paysage.

  > En prenant l'exemple précédent de $\mathcal{H}=\{h(x)=ax^{p}| a\in \mathbb{R}, p\in\mathbb{Z}\}$, pour certaines données observées données $(x_1,y_1),(x_2, y_2),\ldots,(x_n,y_n)$, la perte empirique s'écrit explicitement $$ \mathcal{L}(a, p) = \frac{1}{n}\sum_{i=1}^n L(ax_i^p, y_i) \; .$$ En prenant la valeur de $\mathcal{L}(a,p)$ comme la hauteur associée à une coordonnée de localisation $(a,p)$, on obtient un paysage de "montagnes" et "vallées" défini sur le domaine de $(a,p)$

- L'**algorithme d'apprentissage** est le schéma computationnel spécifique pour rechercher les _paramètres_ optimaux dans l'espace d'hypothèses choisi. Pour l'espace d'hypothèses dans l'exemple précédent avec la forme $h(x)=ax^p$, un algorithme d'apprentissage est une série d'opérations computationnelles concrètes qui, lorsqu'il se termine, retourne la "localisation" $(a,p)$ au fond d'une "vallée" dans le paysage de la **perte empirique** (qui idéalement se trouve aussi au fond du paysage de la **perte attendue**). 

  Le processus de recherche de la fonction optimale dans l'espace d'hypothèses en utilisant un algorithme d'apprentissage est appelé **entraînement**, et les paires caractéristique-cible observées impliquées dans ce processus sont appelées **ensemble d'entraînement**.

  Lorsqu'une solution analytique n'est pas possible, une boucle itérative pour approcher les paramètres optimaux doit être invoquée dans l'**algorithme d'apprentissage**. Dans ce cas, un algorithme d'apprentissage peut être vu comme un _système dynamique_ dans l'espace des paramètres. 

  > Un _système dynamique_ est un ensemble de règles pour mettre à jour un état basé uniquement sur l'état actuel. Prenons $h(x)=ax^p$ comme exemple, une hypothèse (fonction) est complètement déterminée par la paire $(a, p)$. Un système dynamique sur $(a,p)$ varie $a$ et $p$ uniquement basé sur $(a,p)$.  Par exemple, à chaque étape de mise à jour, nous avons l'évolution $a\rightarrow a+ \Delta a,\; p\rightarrow p + \Delta p$ où $\Delta a = (a^2 + 3p)\times \ell$ et $\Delta p = (-a+p)\times \ell$ avec $\ell$ définissant la magnitude de chaque incrément. Notez que les incréments $(\Delta a, \Delta p)$ sont uniquement déterminés par l'état actuel $(a,p)$. Ainsi, une position initiale $(a,p)$ dessine une trajectoire après plusieurs étapes de mise à jour. En particulier, lorsque $\ell\rightarrow 0$, on aboutit à un système d'équations différentielles.

  Un algorithme d'apprentissage est un tel système dynamique avec un ensemble de règles de mise à jour qui déplace l'état vers une position plus basse dans le paysage de la perte empirique (par exemple $\mathcal{L}(a,p)$) et s'arrête finalement au fond d'une vallée. La **descente de gradient** est l'idée fondamentale sous-jacente à la plupart des algorithmes d'apprentissage traitant de la **perte empirique** en apprentissage automatique supervisé.

#### **Exercice 2.3.4 Descente de gradient en 1 dimension**

En supposant que nous recherchons la fonction optimale dans l'espace d'hypothèses $\mathcal{H} = \{h(x)=kx|k\in\mathbb{R}\}$. C'est la pente optimale $k$ pour minimiser une certaine perte empirique. Supposons également que la perte empirique est donnée par $\mathcal{L}(k) = k^2-5k+6$, c'est-à-dire un paysage défini sur un espace à 1 dimension. Nous recherchons le modèle optimal identifié par un certain $k^*$. 

1. Comment trouver analytiquement le $k^*$ optimal pour cette perte empirique ? Quel est le résultat ?
2. Quelle est la dérivée de $L$ par rapport à $k$ ?
3. Quel est le signe de la dérivée lorsque $k$ est plus petit que le $k^*$ optimal ? et lorsque $k>k^*$ ?
4. Comment la magnitude de la dérivée varie-t-elle lorsque $k$ approche $k^*$ depuis la gauche ? et depuis la droite ?
5. Établissez une règle pour mettre à jour $k$ avec une petite magnitude d'incrément $\ell$, de sorte que où que soit $k$, l'incrément sera dans la direction pour approcher $k^*$ depuis le $k$ actuel.
6. Comment rendre la règle d'incrément adaptative de sorte que l'incrément "ralentisse" à chaque mouvement lorsque $k$ se rapproche de $k^*$ ? Indice : magnitude de la dérivée.
7. Implémentez cet algorithme d'apprentissage avec Python avec l'aide des indications dans la cellule de code ci-dessous. 
8. Tracez $k$ en fonction des étapes d'itération jusqu'à ce que $k$ devienne plus ou moins stable. Faites des essais avec différentes initialisations de $k$ et différentes valeurs de $\ell$. 
9. Zoomez sur la partie finale de $k$ versus l'étape d'itération, que voyez-vous pour un très grand nombre d'itérations totales ?
9. Quel est l'effet de $\ell$ ? Indice : en termes d'étapes pour converger, et en termes de précision vers $k^*$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Définir la fonction de perte empirique
def empirical_loss(k):
    return k**2 - 5*k + 6

# Définir la dérivée de la perte empirique
def derivative_loss(k):
    pass

# Définir la règle de mise à jour
def update_rule(k, l):
    pass

# Implémenter l'algorithme d'apprentissage
def learning_algorithm(k, l, num_steps):
    record_k = [k]
    for i in range(num_steps):
        k = update_rule(k, l)
        record_k.append(k)
    return np.array(record_k)

# Définir les paramètres initiaux
k = 0

# Définir la magnitude de l'incrément
l = 0.1

# Définir le nombre d'étapes
num_steps = 100

# Exécuter l'algorithme d'apprentissage

# Tracer k en fonction des étapes d'itération

#### **Exercice 2.3.3 Descente de gradient dans la vie réelle**

Vous êtes largué aléatoirement d'un hélicoptère quelque part dans les Alpes. Votre objectif est d'arriver au point le plus bas à proximité.

1. Qu'avez-vous réellement fait pour atteindre cet objectif ? Décrivez explicitement votre processus de prise de décision. (Ignorez les détails tels que les petits obstacles, les plantes et considérez le paysage comme lisse)
2. Considérez le paysage comme lisse, et supposez que vous ne pouvez voir que votre environnement, comment optimiser chaque pas localement pour suivre le chemin le plus court vers le point le plus bas à proximité ?

### 2.4 Mini régression linéaire avec "scikit-learn"

Maintenant que nous avons clarifié la tâche d'apprentissage automatique pour le problème du projectile et exploré les données d'entraînement, il est temps de choisir un **espace d'hypothèses**, définir une **fonction de perte** et implémenter un **algorithme d'apprentissage** pour obtenir le **modèle entraîné**, c'est-à-dire une fonction optimale qui peut faire des prédictions intelligentes sur la distance d'un projectile avec la connaissance de l'état de lancement dans de nouvelles expériences.

Pour tester la performance du modèle entraîné $h^*(\cdot)$, nous avons construit l'**ensemble de données de test** dans `test_set_1.dat` et utilisons la perte empirique de $h^*(\cdot)$ appliquée sur cet ensemble de test comme une enquête pour la vraie performance, c'est-à-dire la perte attendue. Pour que cette enquête soit fidèle, il est crucial de garantir que l'**ensemble de test** et l'**ensemble d'entraînement** ne partagent aucun point de données (paires caractéristique-cible), sinon la performance réelle pourrait être surestimée. 

Pour avoir un aperçu de la façon dont l'apprentissage automatique fonctionne, nous allons utiliser la bibliothèque "scikit-learn" (`import sklearn`) pour réaliser l'apprentissage automatique d'une simple "régression des moindres carrés ordinaires", ce qui signifie 

- Pour l'**espace d'hypothèses**, nous choisissons un modèle linéaire simple qui est $$\mathcal{H}=\{ h(v_x, v_y) = w_x v_x + w_y v_y + w_0| (w_x, w_y, w_0) \in \mathbb{R}^3\}\;.$$ Par conséquent, l'entraînement se termine par l'optimisation du triplet de paramètres $(w_x, w_y, w_0)$ par rapport à la perte empirique. Les coefficients $(w_0, w_x, w_y)$ sont appelés "poids" dans un modèle linéaire.
- Pour la **perte empirique**, nous choisissons la **fonction de perte** comme $$L(\hat d, d ) = (\hat d - d)^2$$ où $\hat d$ et $d$ sont respectivement les distances prédites et réelles.
- L'hypothèse optimale pour le réglage ci-dessus peut être résolue analytiquement par inversion de matrice, c'est-à-dire que l'**algorithme d'apprentissage** est réalisé via l'inversion numérique de matrice.

#### **Exercice 2.4.1**

1. En tenant compte des résultats de l'exploration des données sur la dépendance de $d$ par rapport à $(v_x, v_y)$, que pouvez-vous deviner sur les signes de $w_x$ et $w_y$ d'une fonction linéaire optimale ?

2. Écrivez l'expression de la perte empirique, si $n$ paires caractéristique-cible sont données.

#### Régression des Moindres Carrés Ordinaires (OLS) simple avec `sklearn`

Nous utilisons les données dans `training_set_1.dat` pour l'entraînement et `test_set_1.dat` pour évaluer le modèle entraîné. 

La cellule de code suivante réalise l'ensemble du processus de régression OLS. Lisez, exécutez et jouez avec le code, pour répondre aux questions de l'exercice suivant.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

train1 = np.loadtxt('./data/projectile_1/training_set_1.dat')
test1 = np.loadtxt('./data/projectile_1/test_set_1.dat')

# Utiliser les données d'entraînement (vx, vy) pour prédire d
X_train = train1[:, :2]
y_train = train1[:, 2]
X_test = test1[:, :2]
y_test = test1[:, 2]

# Initialiser le modèle de régression OLS
ols = LinearRegression()

# afficher les poids avant l'entraînement
print("Before training ------------")
try:
    print("Coefficients before training:", ols.coef_)
    print("Intercept before training:", ols.intercept_)
except:
    print("Weights before training: not available")

# Entraîner le modèle de régression OLS
ols.fit(X_train, y_train)

# afficher les poids après l'entraînement
print("After training ------------")
try:
    print("Coefficients after training:", ols.coef_)
    print("Intercept after training:", ols.intercept_)
except:
    print("Weights after training: not available")

# Prédire sur l'ensemble de test
y_pred = ols.predict(X_test)

# Calculer la mesure de performance (erreur quadratique moyenne)
mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error (on test set):", mse)

%matplotlib widget
plt.clf()
plt.scatter(y_test, y_pred, s=20, c='b', marker='o', alpha=0.5)
plt.plot([100, 3000], [100, 3000], 'k--')
plt.show()

# Implémenter la perte empirique
def empirical_loss(X, y, model):
    pass

    # à compléter

#### **Exercice 2.4.2** Régression OLS avec `sklearn`

1. Quel est le nom de référence (ou objet) qui porte les informations sur l'espace du modèle linéaire, la perte empirique et la méthode d'entraînement ? Comment cet objet est-il créé par le code ?
2. Après que les ensembles de données d'entraînement et de test sont chargés et préparés, quels sont les deux appels / étapes cruciaux pour obtenir un modèle linéaire simple entraîné ?
3. Quelle ligne de code porte le processus d'entraînement réel ? 
4. Comment les données d'entraînement sont-elles fournies au processus d'entraînement ? Important, que représentent les lignes et les colonnes dans `X_train` et `y_train` ? Que se passe-t-il si nous avons $M$ paires caractéristique-cible où chaque caractéristique compresse $st$ variables d'entrée, et chaque cible compresse $t$ variables ?
5. Après l'entraînement, quelles valeurs les poids prennent-ils pour $w_x, w_y, w_0$ respectivement ?
6. À quoi devrait ressembler le graphique si le modèle entraîné fait des prédictions parfaites ?
7. Complétez la fonction `empirical_loss` prenant `X`, `y` paires caractéristique-cible dans le même format que `y_test`, `y_pred` et un objet modèle `model` comme arguments pour retourner la perte empirique, sans utiliser `mean_squared_loss`. Vérifiez qu'elle retourne le même résultat que `mse`.
8. Utilisez `empirial_loss` pour évaluer le modèle entraîné sur l'ensemble d'entraînement. Comment cela se compare-t-il à la perte empirique sur l'ensemble de test ? 


#### **Exercice 2.4.3** Créez votre propre modèle linéaire avec une `class`

La classe est un concept fondamental en Python, car en Python tout est un objet d'un certain "type". Ce "type" fait référence à une "classe". Par exemple, exécutez la cellule de code suivante, et vous verrez que tous les types de données intégrés sont certaines classes.

In [ ]:
a = int(0)
print(type(a))

b = float(0.0)
print(type(b))

c = str('0')
print(type(c))

d = list([0])
print(type(d))

e = list()
print(type(e))


Dans la cellule de code ci-dessus, chaque nom de variable (également appelé "référence") représente un objet ou une instance d'une certaine classe. Par exemple, `a` est une instance ou un objet de la classe `int`, `d` et `e` sont deux objets différents ou instances de la même classe `list`. 

Une classe est un plan ou une méta-forme pour créer des objets concrets. Voici un morceau de code pour déclarer une classe `my_linear_model`

In [ ]:
class my_linear_model:
    def __init__(self, w_x=0.0, w_y=0.0, w_0=0.0):
        self.w_x = w_x
        self.w_y = w_y
        self.w_0 = w_0
    
    def predict(self, X):
        assert X.ndim == 2 and X.shape[1] == 2, "X must be a 2D array with 2 columns"
        return self.w_x*X[:, 0] + self.w_y*X[:, 1] + self.w_0

my_model1 = my_linear_model()
my_model2 = my_linear_model(1.0, 1.0, 1.0)

X = np.array([[1.0, 2.0], [2.0, 3.0], [3.0, 4.0]])

y_pred1 = my_model1.predict(X)
y_pred2 = my_model2.predict(X)

print('my_model1.predict(X)', y_pred1)
print('my_model2.predict(X)', y_pred2)


1. Créez une instance `my_traine_model` de la classe `my_linear_model` avec les poids initialisés en utilisant les poids du modèle entraîné.
2. Faites une prédiction en utilisant cette instance avec `X_test`, et comparez avec la prédiction du modèle entraîné `ols`.

### 2.5 Modèles linéaires et ingénierie des caractéristiques

#### 2.5.1 Modèle linéaire

Un **modèle linéaire** est l'un des types de modèles d'apprentissage automatique les plus simples et les plus fondamentaux.
Il suppose que la **variable cible** ( $y$ ) peut être exprimée comme une **combinaison linéaire** des **variables d'entrée** ( $x_1, x_2, \ldots, x_n$ ):

$$
y = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_n x_n 
$$

où :

* $w_0$ est le terme de **biais (ordonnée à l'origine)**,
* $w_1, \ldots, w_n$  sont les **coefficients du modèle (poids)**,

Sous forme vectorielle :

$$
y = \mathbf{w}^\top \mathbf{x} + w_0
$$

Cette formulation s'applique à la fois à la **régression** (prédire des résultats scalaires) et à la **classification** (prédire des catégories, souvent via des fonctions logistiques ou softmax).


> **Pourquoi les modèles linéaires sont importants**
>
> Les modèles linéaires sont conceptuellement simples mais extrêmement puissants :
> 
> * Ils sont **faciles à interpréter** — chaque poids montre directement l'influence d'une caractéristique sur la sortie.
> * Ils sont **computationalement efficaces** — l'entraînement implique de résoudre des problèmes d'optimisation convexes (souvent via les moindres carrés ou la descente de gradient).
> * Ils servent de **modèle de référence** — souvent le premier modèle testé avant des modèles non linéaires plus complexes.
> * De nombreux modèles non linéaires (comme les réseaux de neurones) peuvent être vus comme des compositions de **transformations linéaires plus des non-linéarités**.

#### 2.5.2 Ingénierie des caractéristiques et puissance des modèles linéaires

Because linear models only learn *linear relationships*, the expressiveness of the model depends heavily on the **features** provided. That’s where **feature engineering** becomes critical.

**Feature engineering** means transforming raw feature input variables into informative features that better capture the underlying relationships between inputs and outputs. For linear models, this can include:

* **Polynomial features:** ( $x, x^2, x^3, \ldots$ ) allow modeling nonlinear trends.
* **Interaction terms:** ( $x_1 \times x_2$ ) capture nonlinearity between features.
* **Normalization/scaling:** ensures all features contribute proportionally (important for gradient-based optimization).
* **Feature selection:** removing redundant or irrelevant variables improves generalization.

With the right feature transformations, a linear model can approximate surprisingly complex patterns.

Formellement, pour une caractéristique de $n$ variables d'entrée $\mathbf{x} = (x_1, x_2, \ldots, x_n)$ (ou aussi communément appelée "$n$ caractéristiques pour une entrée"), l'**ingénierie des caractéristiques** consiste à construire $m$ caractéristiques complexes $\phi_1(\mathbf{x}), \phi_2(\mathbf{x}), \ldots, \phi_m(\mathbf{x})$ à partir des $n$ caractéristiques brutes, avec généralement $m>n$, pour capturer la non-linéarité dans la fonction cible. Chaque $\phi_i(\mathbf{x})$ est une fonction de toutes les variables d'entrée brutes $x_1, x_2, \ldots, x_n$. 

Un modèle linéaire plus expressif possède maintenant $m$ poids et s'écrit 

$$
y = w_1 \phi_1(\mathbf x) + w_2 \phi_2(\mathbf x) + \cdots + w_m \phi_m(\mathbf x) + w_0
$$

ou sous forme vectorielle

$$
y = \mathbf{w}^T\cdot \mathbf{\phi}(\mathbf{x}) + w_0\;.
$$

Ainsi, la régression linéaire est transformée en une nouvelle mais toujours **linéaire** dans les paramètres d'ajustement $\mathbf{w}$.

Par exemple, les **caractéristiques polynomiales** $\phi_p(x)$ pour une seule variable d'entrée $x$ s'écrivent $\phi_p(x) = x^p$. 

#### **Exercice 2.5.1 Plus d'exploration des données pour le projectile**

Nous allons approfondir l'exploration des données avec le `training_set_1.dat`. Utilisez une cellule de code ci-dessous pour investiguer les questions suivantes.

1. La dépendance de la distance $d$ est-elle linéaire sur chacun de $v_x$ et $v_y$, lorsque l'autre est fixé ?
2. Comment la dépendance de $d$ sur $v_x$ pour un $v_y$ fixé varie-t-elle lorsque $v_y$ varie ? 
3. La même question que Q2 sauf en échangeant $v_x$ et $v_y$.
4. Quel type de forme pouvez-vous deviner pour la fonction cible ?
5. Votre supposition est-elle cohérente avec les résultats (en particulier le graphique de dispersion coloré) dans l'exploration des données précédente ?
6. Votre supposition est-elle cohérente avec la forme du nuage de points dans la régression OLS ?

#### **Exercice 2.5.2 Régression OLS avec ingénierie des caractéristiques pour le projectile**

Basé sur les résultats OLS précédents et l'analyse ci-dessus, une forme linéaire simple $h(v_x, v_y) = w_x v_x + w_y v_y + w_0$ ne capture pas la complexité de la fonction cible. Nous allons transformer les caractéristiques brutes $v_x,\; v_y$ en caractéristiques d'ordre supérieur. Chaque nouvelle caractéristique $\phi_{p,q}$ est de la forme $\phi_{p,q} = v_x^pv_y^q$, avec $p+q$ de $1$ jusqu'à $3$.

1. Pour $p+q=1, 2, 3$, listez toutes les caractéristiques possibles $\phi$. De cette façon, nous transformons 2 variables d'entrée en une caractéristique de $m$ variables, quelle est $m$ ?
2. Utilisez une cellule de code pour charger et préparer `X_train`, `y_train`, `X_test`, `y_test` comme la régression OLS, puis construisez, en utilisant le même format que `X_train` ou `X_test`, `Xnew_train` et `Xnew_test` des nouvelles caractéristiques. Gardez une trace de quelle colonne dans la matrice d'entrée correspond à quelle paire $(p,q)$.
3. Refaites la régression linéaire, cette fois avec un objet modèle linéaire `ols_new`, avec les nouvelles caractéristiques.
4. Faites un graphique de dispersion de $y_pred_new$ versus $y_test$, dans la même figure pour comparer avec OLS.
5. Évaluez la perte empirique de la cible nouvellement prédite sur l'ensemble de données de test. Comment cela se compare-t-il à l'OLS précédent ? Est-ce cohérent avec les changements de forme du nuage de points ?
6. Investiguez les coefficients pour chaque index de nouvelle caractéristique par $(p,q)$. Comparez avec les coefficients dans le cas sans ingénierie des caractéristiques, quelle est l'origine de l'amélioration ?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib widget

train1 = np.loadtxt('./data/projectile_1/training_set_1.dat')
test1 = np.loadtxt('./data/projectile_1/test_set_1.dat')

# Utiliser les données d'entraînement (vx, vy) pour prédire dx
X_train = train1[:, :2]
y_train = train1[:, 2]
X_test = test1[:, :2]
y_test = test1[:, 2]

# Construire les nouvelles caractéristiques
# X_train_new
# X_test_new

# Initialiser le modèle de régression OLS
# ols = 
# ols_new = 

# Entraîner le modèle de régression OLS avec à la fois les caractéristiques brutes et les nouvelles caractéristiques

# Prédire sur l'ensemble de test
# y_pred = 
# y_pred_new = 

# Calculer la mesure de performance (erreur quadratique moyenne)
mse = mean_squared_error(y_test, y_pred)
mse_new = mean_squared_error(y_test, y_pred_new)

print("Erreur quadratique moyenne     (sur l'ensemble de test):", mse)
print("Erreur quadratique moyenne new (sur l'ensemble de test):", mse_new)

# Examiner les coefficients du modèle pour les nouvelles caractéristiques

# Nuage de points de y_pred_new en fonction de y_test

### 2.6 Généralisation

Jusqu'à présent, nous avons discuté des éléments clés pour obtenir un modèle d'apprentissage automatique raisonnablement optimisé pour une tâche donnée. Ici, nous allons explorer les facteurs dans le processus d'entraînement qui peuvent affecter la **généralisation**.

D’un point de vue pratique, la **généralisation** correspond à la performance d’un modèle sur des données jamais vues — c’est-à-dire la perte empirique sur de nouveaux exemples (inconnus). Conceptuellement, la **généralisation** représente la *perte attendue* sur l’ensemble de la *population* des couples caractéristique–cible. Ici, le terme *population* fait référence à la distribution de probabilité sous-jacente des variables ou objets considérés.

* La difficulté d'obtenir une bonne **généralisation** provient du fait que l'entraînement est toujours réalisé sur un ensemble de données fini, qui diffère inévitablement de la vraie **population**. Par conséquent, l'entraînement produit un modèle optimisé pour la perte empirique sur les données d'entraînement, qui est, en principe, différente du modèle qui minimiserait la **perte attendue** sur la vraie population. Ainsi, pour exploiter pleinement le potentiel d'un espace d'hypothèses donné, la meilleure approche consiste à utiliser autant de données que possible.

  Mais quelle quantité de données est suffisante ? La réponse est que nous ne le savons pas vraiment, car la distribution de probabilité sous-jacente est inconnue. C’est précisément pourquoi la collecte de données est nécessaire pour entraîner un modèle. Sans connaître la véritable distribution, il est difficile de déterminer la quantité de données nécessaire pour atteindre un niveau de tolérance à l’incertitude souhaité.

* Il existe une autre couche de difficulté pour obtenir une bonne **généralisation**. Un espace d’hypothèses possède un certain degré de *flexibilité* ou d’*expressivité*. Par exemple, un modèle linéaire ( $h(x) = w_1x + w_0$ ) est moins flexible qu’un modèle quadratique ( $h(x) = w_2x^2 + w_1x + w_0$ ) ; ce dernier a un pouvoir expressif plus grand. La flexibilité, ou l’expressivité, d’un modèle est aussi appelée sa *complexité*. En principe, on souhaite que le modèle soit suffisamment flexible ou complexe pour apprendre un large éventail de fonctions cibles possibles.

  Cependant, puisqu’un modèle flexible est optimisé sur un ensemble de données d’entraînement fini — qui comporte inévitablement des particularités propres — il peut devenir trop adapté à cet ensemble. Autrement dit, le modèle peut apprendre des caractéristiques aléatoires qui n’existent pas dans la véritable population mais apparaissent dans l’ensemble d’entraînement. Ce phénomène, appelé **surapprentissage**, résulte de la combinaison d’un volume de données limité et de la flexibilité du modèle.


Pour ces raisons, lorsque la quantité de données est limitée, il est courant de diviser l'ensemble de données disponible en un **ensemble d'entraînement** et un **ensemble de test** afin de :

1. utiliser autant de données que possible pour l'entraînement, afin de mieux approximer la vraie population, et
2. réserver un ensemble séparé, exclu de l'entraînement, pour servir d'indicateur de performance réelle et de protection contre le surapprentissage.

#### **Exercice** 2.6.1 L'effet de la taille de l'ensemble d'entraînement

Dans cet exercice, nous allons supposer que la perte empirique sur l'ensemble de données de test `test_set_1.dat` est une mesure relativement bonne de la vraie performance. Jusqu'à présent, nous utilisons l'ensemble de données d'entraînement complet pour entraîner un modèle linéaire ordinaire. L'objectif de cet exercice est d'investiguer comment la taille de l'ensemble de données d'entraînement affecte la vraie performance d'un modèle entraîné. Nous allons utiliser le modèle linéaire avec ingénierie des caractéristiques de la section précédente comme notre espace d'hypothèses.

Utilisez la cellule de code ci-dessous pour les exercices suivants
1. Construisez les caractéristiques avec ingénierie comme le dernier exercice pour l'ensemble d'entraînement et de test.
2. Créez un tableau de tailles d'ensembles de données d'entraînement qui sont également espacées en log en utilisant `numpy.logspace` de 10 à 10000. Ce tableau devrait inclure 9 tailles différentes avec 10 et 1000 inclus.
3. Faites une boucle for itérant sur le tableau de tailles. Pour chaque taille de données d'entraînement, générez un ensemble d'entraînement de la taille correspondante en retirant uniformément aléatoirement de l'ensemble de données d'entraînement original (chargé depuis `training_set_1.dat`), en utilisant `numpy.random.choice`. 
4. Pour chaque ensemble d'entraînement (à l'intérieur de la boucle for) :
  - entraînez un modèle de régression OLS
  - faites une prédiction en utilisant l'entrée de test, évaluez la prédiction de test, et stockez-la dans une liste
  - faites une prédiction en utilisant l'entrée d'entraînement, évaluez la prédiction d'entraînement, et stockez-la dans une liste
  - stockez les poids et l'ordonnée à l'origine dans une liste
5. Tracez la perte d'entraînement et la perte "vraie" en fonction des tailles de l'ensemble de données d'entraînement. Mettez les tailles en échelle logarithmique. 
6. Tracez les poids et les ordonnées à l'origine pour différentes tailles d'ensembles de données d'entraînement.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib widget

train1 = np.loadtxt('./data/projectile_1/training_set_1.dat')
test1 = np.loadtxt('./data/projectile_1/test_set_1.dat')

X_train = train1[:, :2]
y_train = train1[:, 2]
X_test = test1[:, :2]
y_test = test1[:, 2]

# Feature engineering
X_train_new = np.zeros((len(X_train), 9))
X_test_new = np.zeros((len(X_test), 9))

# Training set size array
n_train = np.logspace(1, 3, 9).astype(int)

Parameters = []
Training_loss = []
Testing_loss = []
# Training loop
for i, n in enumerate(n_train):
    # Generate training set
    idx = np.random.choice( replace=False)
    X_train_sub = X_train_new[idx]
    y_train_sub = y_train[idx]
    
    # Train OLS regression model
    
    # Make a prediction using the testing input
    
    Testing_loss.append(mean_squared_error(y_test, y_pred_test))
    
    # Make a prediction using the training input
    
    Training_loss.append(mean_squared_error(y_train_sub, y_pred_train_sub))
    
    # Store the weights and the intercept
    Parameters.append(ols.coef_)

# Plot the training loss and "true" loss as a function of the training dataset sizes
plt.figure()
plt.clf()

plt.xscale('log')
plt.legend()
plt.show()

# Plot the weights and the intercept as a function of the training dataset sizes
plt.figure()
plt.clf()

# plt.xscale('log')
plt.legend()
plt.show()


#### **Lutter contre le surapprentissage avec la Régularisation**

Étant donné le risque de surapprentissage dû à la combinaison d'un ensemble d'entraînement fini et de flexibilité du modèle, nous introduisons un terme de **régularisation** (également appelé "pénalité") dans l'objet d'optimisation. La pénalité est une fonction notée $\mathcal{L}_R()$ mappant les paramètres d'ajustement du modèle d'apprentissage automatique vers un scalaire. 

La propriété de ce terme de pénalité est qu'il favorise les paramètres du modèle qui rendent la fonction du modèle moins complexe ou "sans caractéristiques", ou simplement simple. Par exemple, une forme de pénalité courante pour un modèle linéaire $y=\sum_{i=1}^n w_i x_i + w_0$ est la forme quadratique $\sum_{i=1}^n w_i^2$, qui favorise les poids à être petits lorsqu'un algorithme d'apprentissage recherche pour le minimiser. 

Avant, l'objet d'optimisation était toujours uniquement la perte empirique sur l'ensemble d'entraînement. Avec un terme de pénalité, l'objet d'optimisation devient $$ \text{Perte empirique} + \alpha \text{Pénalité} $$ où $\alpha$ est introduit comme un _hyper-paramètre_ appelé "force de pénalité", pour définir le pouvoir d'influence de la pénalité sur le processus d'optimisation. Pour une régression linéaire utilisant la perte au carré, l'objet d'optimisation devient 

$$
\tilde{ \mathcal{L} }= \frac{1}{N} \sum_{i=1}^N (\mathbf{w}^T\cdot \mathbf{x}_i + w_0 - y_i)^2 + \alpha \mathbf{w}^T\cdot \mathbf{w}
$$

où $N$ est le nombre de points de données. Lorsque $\alpha$ est très grand, tous les poids tendent à être zéro, et lorsque $\alpha$ est petit, cela récupère la régression des moindres carrés ordinaires. Notez également que $\alpha$ ainsi défini représente le pouvoir de pénalité par point de données. Cette forme de régression linéaire est appelée "régression Ridge".

#### **Exercice 2.6.2 Régression Linéaire Ridge**

La régression linéaire Ridge est également implémentée par scikit-learn. Voici la façon de l'appeler avec un certain $\alpha$ :
```
from sklearn.linear_models import Ridge
ridge_model = Ridge(alpha=1.0)
```
Le reste reste le même que la régression OLS. Notez cependant que dans l'implémentation scikit-learn, le réglage d'alpha n'est pas normalisé par la taille d'entraînement, ce qui signifie que pour avoir une force de pénalité similaire pour différentes tailles d'échantillons d'entraînement $N$, le `alpha` dans `Ridge()` devrait être défini comme $N\times alpha$.

D'après l'exercice précédent, la taille d'échantillon d'entraînement $100$ montre un degré modéré de surapprentissage. Nous allons mettre en place un échantillon d'entraînement de 100 points de données et utiliser cet ensemble d'entraînement pour effectuer la régression Ridge pour différentes valeurs de $\alpha$ pour voir son effet.

1. Mettez en place un échantillon d'entraînement de 100 échantillons tirés aléatoirement de l'ensemble d'entraînement original et faites l'ingénierie des caractéristiques pour obtenir `X_train_new` et `X_test_new`.
2. Construisez une série de valeurs de $\alpha$ allant de $10^{-5}$ à $10^{4}$ avec un espacement en échelle logarithmique, appelez-la `Alphas`. Suggestion : 37 valeurs avec $10^{-5}$ et $10^4$ inclus.
3. Bouclez sur différentes valeurs de $\alpha$, et dans chaque boucle :
  - Effectuez la régression linéaire Ridge avec une force de pénalité normalisée $\alpha$, c'est-à-dire `Ridge(alpha=N*Alphas[i])`
  - Faites une prédiction avec les données de test, évaluez le résultat et stockez-le dans une liste.
  - Faites une prédiction avec les données d'entraînement, évaluez les résultats et stockez-les dans une liste.
  - Stockez les poids ajustés et l'ordonnée à l'origine dans une liste
4. Tracez les poids en fonction de $\alpha$, en utilisant l'échelle logarithmique pour $\alpha$.
5. Tracez la perte d'entraînement et la perte de test en fonction de $\alpha$.
6. Quelle valeur de $\alpha$ est optimale ? Pourquoi ? 
7. Quels sont les poids à l'$\alpha$ optimal ? Quelle est l'interprétation physique ?

Utilisez la cellule de code suivante.

In [ ]:
# Exercise 2.6.2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

%matplotlib widget

train1 = np.loadtxt('./data/projectile_1/training_set_1.dat')
test1 = np.loadtxt('./data/projectile_1/test_set_1.dat')

X_train = train1[:, :2]
y_train = train1[:, 2]
X_test = test1[:, :2]
y_test = test1[:, 2]

# Feature engineering
X_train_new = np.zeros((len(X_train), 9))
X_test_new = np.zeros((len(X_test), 9))

# Construct a training set of 100 samples
N = 100
idx = np.random.choice(len(X_train_new), N, replace=False)
X_train_sub = X_train_new[idx]
y_train_sub = y_train[idx]

# Set up a series of alpha values
Alphas = np.logspace(-7, 7, 41)

Training_loss = []
Testing_loss = []
Weights = []
Intercepts = []
for i, alpha in enumerate(Alphas):
    # Perform Ridge regression
    
    
    # Make a prediction with the test data
    
    
    # Evaluate the result and store it into a list
    
    
    # Make a prediction with the training data
    
    
    # Evaluate the result and store it into a list
    
    
    # Store the fitted weights and intercept
    Weights.append(ridge_model.coef_)
    Intercepts.append(ridge_model.intercept_)

# Plot the weights as a function of alpha
Weights = np.array(Weights)
plt.figure()
plt.clf()

plt.xscale('log')
plt.legend()
plt.show()

# Plot the training loss and the test loss as function of alpha
plt.figure()
plt.clf()
plt.plot(Alphas, Training_loss, label='Training loss')
plt.plot(Alphas, Testing_loss, label='Testing loss')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

## 2. Projet-2 : distance horizontale à partir de la norme de vitesse et de l'angle $(v,\theta)$

Appliquez les concepts et techniques revus dans le Projet-1 au Projet-2 : prédire la distance horizontale en utilisant la magnitude de la vitesse initiale ($v$) et l'angle de lancement ($\theta$). 

Combinez du texte markdown et des cellules de code pour réaliser ce projet avec l'aide de scikit-learn. Voici une structure suggérée pour procéder :

- Définir la fonction cible.
- Explorer les données `training_set_2.dat` et `test_set_2.dat`.
- Définir un modèle linéaire avec une ingénierie des caractéristiques appropriée en utilisant les indications de l'exploration des données.
- Effectuer la régression linéaire Ridge
- Trouver l'hyperparamètre optimal, et les poids associés
- Donner une interprétation physique des résultats d'apprentissage automatique.

## Suppléments

In [ ]:
## code for generating training and test sets ./data/projectile_1
g = 9.81

def distance_vx_vy(vx, vy):
    return vx*(vy/g)*2
    
def distance_v_alpha(v, alpha):
    return v*np.sin(2*alpha)*(v/g)

n_train = 10000
n_test = 2000

vx = np.random.normal(80, 15, n_train + n_test)    
vy = np.random.normal(80, 15, n_train + n_test)

v = np.sqrt(vx**2 + vy**2)
alpha = np.arctan2(vy, vx)

dx_std = 10
dx = vx*(vy/g)*2 + np.random.normal(0, dx_std, n_train + n_test)


train1 = np.zeros((n_train, 3))
train1[:,0] = vx[:n_train]
train1[:,1] = vy[:n_train]
train1[:,2] = dx[:n_train]

test1 = np.zeros((n_test, 3))
test1[:,0] = vx[n_train:n_train+n_test]
test1[:,1] = vy[n_train:n_train+n_test]
test1[:,2] = dx[n_train:n_train+n_test]

train2 = np.zeros((n_train, 3))
train2[:,0] = v[:n_train]
train2[:,1] = alpha[:n_train]
train2[:,2] = dx[:n_train]

test2 = np.zeros((n_test, 3))
test2[:,0] = v[n_train:n_train+n_test]
test2[:,1] = alpha[n_train:n_train+n_test]
test2[:,2] = dx[n_train:n_train+n_test]


# np.savetxt('./data/projectile_1/training_set_1.dat', train1, header='vx vy dx')
# np.savetxt('./data/projectile_1/training_set_2.dat', train2, header='v alpha dx')
# np.savetxt('./data/projectile_1/test_set_1.dat', test1, header='vx vy dx')
# np.savetxt('./data/projectile_1/test_set_2.dat', test2, header='v alpha dx')

In [ ]:
import matplotlib.pyplot as plt

sc = plt.scatter(vx[:1000], vy[:1000], c=dx[:1000], marker='o')
plt.colorbar(sc, label='dx')
plt.xlabel('vx')
plt.ylabel('vy')
plt.title('vy vs vx colored by dx (first 1000 points)')



In [ ]:
plt.plot(vx, dx, 'o', alpha=0.2, mec='none' )
plt.xscale('log')
plt.yscale('log')
plt.xlabel('vx')
plt.ylabel('dx')
plt.title('dx vs vx')
plt.show()


In [ ]:
x = np.linspace(0, 2, 101)
y1 = 2*x
y2 = 2*x**2
z = 2*x**1.5

plt.plot(x, (y1-z)**2, label='y1')
plt.plot(x, (y2-z)**2, label='y2')
plt.legend()

s1 = np.random.exponential(0.1,1000)
t1 = 2*s1**1.5 
s2 = np.random.exponential(1.0,1000)
t2 = 2*s2**1.5

plt.plot(s1, t1, 'o', alpha=0.2, mec='none')
plt.plot(s2, t2, 'o', alpha=0.2, mec='none')
plt.show()

s = s1
t = t1
h1 = 2*s
h2 = 2*s**2
plt.figure()
plt.plot(t, h1, 'o', alpha=0.2, mec='none')
plt.plot(t, h2, 'o', alpha=0.2, mec='none')
plt.show()

print('h1', np.mean(np.abs(h1-t)))
print('h2', np.mean(np.abs(h2-t)))

d1 = np.zeros((len(s1), 2))
d2 = np.zeros((len(s2), 2))
d1[:,0] = s1
d1[:,1] = t1
d2[:,0] = s2
d2[:,1] = t2
# np.savetxt('./data/projectile_1/population_1.dat', d1, header='x y')
# np.savetxt('./data/projectile_1/population_2.dat', d2, header='x y')



In [ ]:
## exercise 2.6.1
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib widget

train1 = np.loadtxt('./data/projectile_1/training_set_1.dat')
test1 = np.loadtxt('./data/projectile_1/test_set_1.dat')

X_train = train1[:, :2]
y_train = train1[:, 2]
X_test = test1[:, :2]
y_test = test1[:, 2]

# Feature engineering
X_train_new = np.zeros((len(X_train), 9))
X_test_new = np.zeros((len(X_test), 9))

ci = 0
for s in range(1, 4):
    for p in range(s+1):
        X_train_new[:, ci] = X_train[:, 0]**(s-p) * X_train[:, 1]**p
        X_test_new[:, ci] = X_test[:, 0]**(s-p) * X_test[:, 1]**p
        # print('s.{} p.{}, q.{}'.format(s, p, s-p))
        ci += 1

# Training set size array
n_train = np.logspace(1, 3, 9).astype(int)

Parameters = []
Training_loss = []
Testing_loss = []
# Training loop
for i, n in enumerate(n_train):
    # Generate training set
    idx = np.random.choice(len(X_train_new), n, replace=False)
    X_train_sub = X_train_new[idx]
    y_train_sub = y_train[idx]
    
    # Train OLS regression model
    ols = LinearRegression()
    ols.fit(X_train_sub, y_train_sub)
    
    # Make a prediction using the testing input
    y_pred_test = ols.predict(X_test_new)
    Testing_loss.append(mean_squared_error(y_test, y_pred_test))
    
    # Make a prediction using the training input
    y_pred_train_sub = ols.predict(X_train_sub)
    Training_loss.append(mean_squared_error(y_train_sub, y_pred_train_sub))
    
    # Store the weights and the intercept
    Parameters.append(ols.coef_)

# Plot the training loss and "true" loss as a function of the training dataset sizes
plt.figure()
plt.clf()
plt.plot(n_train, Training_loss, label='Training loss')
plt.plot(n_train, Testing_loss, label='Testing loss')
plt.xscale('log')
plt.legend()
plt.show()

# Plot the weights and the intercept as a function of the training dataset sizes
plt.figure()
plt.clf()
pmt = np.array(Parameters)
for i in range(pmt.shape[1]):
    plt.plot(n_train, pmt[:, i], label='w_{}'.format(i+1))
plt.legend()
plt.show()


In [ ]:
# Exercise 2.6.2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

%matplotlib widget

train1 = np.loadtxt('./data/projectile_1/training_set_1.dat')
test1 = np.loadtxt('./data/projectile_1/test_set_1.dat')

X_train = train1[:, :2]
y_train = train1[:, 2]
X_test = test1[:, :2]
y_test = test1[:, 2]

# Feature engineering
X_train_new = np.zeros((len(X_train), 9))
X_test_new = np.zeros((len(X_test), 9))

ci = 0
for s in range(1, 4):
    for p in range(s+1):
        X_train_new[:, ci] = X_train[:, 0]**(s-p) * X_train[:, 1]**p
        X_test_new[:, ci] = X_test[:, 0]**(s-p) * X_test[:, 1]**p
        # print('s.{} p.{}, q.{}'.format(s, p, s-p))
        ci += 1

N = 100
idx = np.random.choice(len(X_train_new), N, replace=False)
X_train_sub = X_train_new[idx]
y_train_sub = y_train[idx]


Alphas = np.logspace(-7, 7, 41)

Training_loss = []
Testing_loss = []
Weights = []
Intercepts = []
for i, alpha in enumerate(Alphas):
    ridge_model = Ridge(alpha=N*alpha)
    ridge_model.fit(X_train_sub, y_train_sub)
    y_pred_test = ridge_model.predict(X_test_new)
    y_pred_train = ridge_model.predict(X_train_sub)
    Training_loss.append(mean_squared_error(y_train_sub, y_pred_train))
    Testing_loss.append(mean_squared_error(y_test, y_pred_test))
    Weights.append(ridge_model.coef_)
    Intercepts.append(ridge_model.intercept_)

Weights = np.array(Weights)
plt.figure()
plt.clf()
for i in range(Weights.shape[1]):
    plt.plot(Alphas, Weights[:,i], label='w{}'.format(i))
# plt.plot(Alphas, Intercepts, label='Intercepts')
plt.xscale('log')
plt.legend()
plt.show()

plt.figure()
plt.clf()
plt.plot(Alphas, Training_loss, label='Training loss')
plt.plot(Alphas, Testing_loss, label='Testing loss')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()


